In [ ]:
%load_ext autoreload
%autoreload 2

root = r"C:\Users\zwecher\Documents\mygit\new_predictor"
#predictor_path  = r"C:\Users\zwecher\Documents\mygit\Indoors_Capabilities\predictor"
#model_path = r"C:\Users\zwecher\Documents\mygit\Indoors_Capabilities\models"

from importlib import reload  # Not needed in Python 2
import logging
reload(logging)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import models
from tensorflow import keras
import sys, os
sys.path.insert(0,root)
#from predictor import predictor, config, restore
import utils
from predictor import buildings_predictor
utils.set_log()
os.chdir(r"C:\Users\zwecher\Documents\mygit\new_predictor\predictor")

In [ ]:
model = buildings_predictor.get_model()

In [ ]:
buildings_predictor.train_model(model,5)

In [ ]:
a.get_weights()